In [ ]:

from json import *
import numpy as np
with open('cva_data.json','r')as f:
    cva_data=f.read()
cva_data=JSONDecoder().decode(cva_data)
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9999
set_session(tf.Session(config=config))

In [ ]:
fi=[x[2:4] for x in cva_data]
inter=[0]*9
for i in fi:
    inter[int(np.round(i[0])-1)]+=1 
#fi=np.array([x[2] for x in facebook],dtype='float32')
import matplotlib.pyplot as plt
plt.bar(np.arange(9),inter)
plt.show()

In [ ]:
word_dict={'PADDING':[0,999999],'UNK':[1,99999]}

pos_dict={'PADDING':0,'UNK':1}
for i in cva_data:
    tp=i
    for word in tp[0]:
        if not word in word_dict:
            word_dict[str(word)]=[len(word_dict),1]
        else:
            word_dict[str(word)][1]+=1
    for pos in tp[1]:
        if not pos in pos_dict:
            pos_dict[str(pos)]=len(pos_dict)        


In [ ]:
trp=sorted(word_dict.items(), key=lambda d:d[1][1], reverse=True)
word_dict={}
co=0

for i in trp:
    if i[1][1]>=2:
        word_dict[i[0]]=[co,i[1][1]]
        #print(co)
    co+=1
print(len(word_dict))


In [ ]:
import numpy as np
embdict={}
plo=0
import  pickle
with open('/home/wuch/chinese_word_new.bin','r')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    while True:
        ch =  f.readline()
        if not ch:
            break
        k=ch.find(" ")
        word=ch[0:k]
        emb=ch[k+1:-2].split()
        if word in word_dict and len(emb)==300:
            for i in range(len(emb)):
                emb[i]=float(emb[i])
            embdict[str(word)]=emb
        plo+=1
        if plo%10000==1:
            print(plo,len(embdict),word)


In [ ]:
from numpy.linalg import cholesky
print(len(embdict),len(word_dict))
print(len(word_dict))
lister=[0]*len(word_dict)#4600
xp=np.zeros(300,dtype='float32')

cand=[]

for i in embdict.keys():
    lister[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)
#R = cholesky(Sigma)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 300)
lister[0]=np.zeros(300,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)

In [ ]:
all_word=[]
all_word_pos=[]
all_word_label=[]

maxlen=135
count=0
for i in cva_data:
    tp=i
    sent=[]
    pos=[]
    for word in tp[0]:
        if word in word_dict:
            sent.append(word_dict[str(word)][0])
        else:
            sent.append(1)
    for po in tp[1]:
        pos.append(pos_dict[str(po)])   
    if len(sent)<=maxlen:
        count+=1
        sent=sent+[0]*(maxlen-len(sent))
        pos=pos+[0]*(maxlen-len(pos))
        all_word.append(sent)
        all_word_pos.append(pos)
        all_word_label.append([tp[2],tp[3]])
print(count)

In [ ]:

import random
all_data_sparse=[]
l1=list(range(2000))
random.shuffle(l1)
all_word0=[]
all_pos0=[]
all_word_label0=[]
count=0
for x in l1:

    all_word0.append(all_word[x])
    all_pos0.append(all_word_pos[x])
    all_word_label0.append(all_word_label[x])
    tt1=[]
    for xx in all_word[x]:
        tp=[0]*xx+[1]+[0]*(5302-xx)
        tt1.append(tp)
    all_data_sparse.append(tt1)
    count+=1
    print(count)


In [ ]:
from __future__ import absolute_import
from recurrentshop import LSTMCell, RecurrentSequential
from seq2seq.cells import *
from seq2seq.models import *
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, TimeDistributed, Bidirectional, Input

In [ ]:
#semi-supervised
result=[]
for fold in range(10):
    labeled_ratio=0.2
    zero_mask=[[1]]*(1800*labeled_ratio)+[[0]]*(1800*(1-labeled_ratio))
    import random
    all_index=range(len(all_data_sparse))
    indexer=random.sample(all_index,1800)
    test_index=list(set(all_index)-set(indexer))
    train_item=np.array(all_word0,dtype='int32')[indexer]
    train_item0=np.array(all_data_sparse,dtype='int32')[indexer]
    train_pos=np.array(all_pos0,dtype='int32')[indexer]
    train_label=np.array(all_word_label0,dtype='float32')[indexer]
    train_mask=np.array(zero_mask,dtype='float32')
    
    test_item0=np.array(all_data_sparse,dtype='int32')[test_index]
    test_item=np.array(all_word0,dtype='int32')[test_index]
    test_pos=np.array(all_pos0,dtype='int32')[test_index]
    test_label=np.array(all_word_label0,dtype='float32')[test_index]
    test_mask=np.array([[1]]*200,dtype='float32')
    print('done')
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import norm
    import tensorflow as tf  
    from keras.layers import Input, Dense, Lambda, Layer
    from keras.models import Model
    from keras import backend as K
    from keras import metrics
    from keras.datasets import mnist
    from keras.layers import GRU,Bidirectional,Multiply
    from keras.models import Sequential,Model
    from keras.layers import Dense, Dropout, Input,Activation,wrappers,Merge,merge,core,advanced_activations
    from keras.layers import LSTM,Conv1D,GlobalMaxPooling1D,SimpleRNN,MaxPooling1D
    from keras.layers import Masking, Embedding,Flatten,RepeatVector,Permute,TimeDistributed
    from keras.regularizers import l2
    from keras.callbacks import ModelCheckpoint,EarlyStopping
    sequenceLength =135
    fmin=0
    #x_train = Input(batch_shape=(None,sequenceLength),dtype='int32')
    x_train2 = Input(batch_shape=(None,sequenceLength),dtype='int32')
    x_train3 = Input(batch_shape=(None,1),dtype='float32')
    #x_train3 = Input(batch_shape=(None,sequenceLength,300),dtype='float32')
    y_train = Input(batch_shape=(None,2),dtype='float32')
    batch_size=50
    latent_dim = 100
    epsilon_std = 1.0
    emb_drop=0.5
    pos_emb_drop=0.5
    
    global_drop=0.5
    
    hid1=100
    xx = Input(batch_shape=(None, sequenceLength,5303),dtype='int32')

    def argmax1(z):
        return K.argmax(z)
    
    # note that "output_shape" isn't necessary with the TensorFlow backend
    zx = Lambda(argmax1, output_shape=(None, sequenceLength))(xx)
    
    emb = Embedding(5303, 300,weights=[lister],input_length=sequenceLength,mask_zero=False)(zx)
    drop_out_emb = Dropout(emb_drop)(emb)
    
    pos_emb = Embedding(52, 50,input_length=sequenceLength,mask_zero=False,dropout=pos_emb_drop)(x_train2)
    drop_out_pos_emb = Dropout(pos_emb_drop)(pos_emb)
    
    merged = merge([drop_out_emb, drop_out_pos_emb], mode='concat',concat_axis=2)
    #merged2 = merge([merged,x_train3], mode='concat',concat_axis=2)


    flat_beta=Bidirectional(LSTM(hid1,return_sequences=True))(merged)
    flat_beta2=Bidirectional(LSTM(hid1,return_sequences=False))(flat_beta)
    flat_betad = Dropout(global_drop)(flat_beta2)

    outer=Dense(2)(flat_betad)

    
    h =LSTM(100,batch_input_shape=(batch_size,sequenceLength, 350),return_sequences=False,dropout_W=0.5,dropout_U=0.5)(merged)

    h_merge=merge([h, outer], mode='concat',concat_axis=-1)
    z_mean = Dense(latent_dim)(h_merge)
    z_log_sigma = Dense(latent_dim)(h_merge)
    
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_sigma / 2) * epsilon
    
    z = Lambda(sampling)([z_mean, z_log_sigma])
    
    
    decoder = RecurrentSequential(decode=True, output_length=sequenceLength,unroll=False, stateful=False)
    decoder.add(Dropout(0.5))
    decoder.add(LSTMDecoderCell(output_dim=100, hidden_dim=100))
    h_decoded = decoder(z)
    
    x_decoded_mean = TimeDistributed(Dense(5303,activation='softmax'))(h_decoded)

    model = Model(input=[x_train2,xx,x_train3], output=[outer,x_decoded_mean])
    def vae_loss(x, x_decoded_mean):

        xent_loss = K.mean(metrics.categorical_crossentropy(x, x_decoded_mean))
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
        return xent_loss+kl_loss

    def myloss(y_true, y_pred):
        return K.mean(x_train3*K.abs(y_pred - y_true), axis=-1)
    

    model.compile(loss=[myloss,vae_loss],loss_weights=[0.5*(1/(labeled_ratio)), 1.],
                  optimizer='rmsprop',metrics=['mae'])
    early=EarlyStopping(monitor='val_loss',patience=10)

    model.fit([train_pos,train_item0,train_mask],[train_label,train_item0], batch_size=batch_size, nb_epoch=50,shuffle=True,validation_split=0.1,callbacks=[early])
    model2 = Model(input=[x_train2,xx,x_train3], output=[outer])
    classes=model2.predict([test_pos,test_item0,test_mask], batch_size=50)

    
    from scipy.stats.stats import pearsonr
    mae=np.mean(np.abs(classes-test_label),axis=0)
    rmse=np.sqrt(np.mean(np.square(classes-test_label),axis=0))
    pcc1=pearsonr(classes[:,0],test_label[:,0])[0]
    pcc2=pearsonr(classes[:,1],test_label[:,1])[0]
    result.append([rmse[0],rmse[1],mae[0],mae[1],pcc1,pcc2])
    print('\n=======Report=======')
    print('rmse:',rmse[0],rmse[1])
    print('mae:',mae[0],mae[1])
    print('pcc:',pcc1,pcc2)
result=np.array(result,dtype='float32')
print('\n=======ave Result=======')
print(np.mean(result,axis=0))
print('\n=======ave Result=======')